In [8]:
%run Fnc_StartupFabricSempy

StatementMeta(, ba97c8d8-4618-4a90-9892-4f106b56a3ce, 20, Finished, Available, Finished)

In [9]:
prm_starttime = datetime.now()
NotebookName = "Nbt_Landing_GetRelations"

StatementMeta(, ba97c8d8-4618-4a90-9892-4f106b56a3ce, 21, Finished, Available, Finished)

In [10]:
Table_Name = 'Landing_Fabric_Relations'

StatementMeta(, ba97c8d8-4618-4a90-9892-4f106b56a3ce, 22, Finished, Available, Finished)

In [11]:
lh_abfs_path = fnc_startUp(Table_Name)

StatementMeta(, ba97c8d8-4618-4a90-9892-4f106b56a3ce, 23, Finished, Available, Finished)

niet leeg


SynapseWidget(Synapse.DataFrame, a4a91146-4ae1-4f26-82ac-3613ab2841d0)

In [12]:
SemanticModels = spark.sql("""select Id, WSID
from Lkh_SemanticLink.Landing_Fabric_Items
where Type='SemanticModel' and DisplayName<>'Report Usage Metrics Model'""")

StatementMeta(, ba97c8d8-4618-4a90-9892-4f106b56a3ce, 24, Finished, Available, Finished)

In [13]:
for Id, WSID in SemanticModels.toLocalIterator():
    dataset_ID = Id
    try:
        relationships = fabric.list_relationships(dataset=dataset_ID, workspace=WSID)
    except Exception as e:
        print(f"Error fetching semantic model relationships for {Id}: {e}")
        continue
    relationships = pd.DataFrame(relationships)
    if not relationships.empty: # check if the list is not empty to avoid errors
        relationships = fnc_PrepareColumns(relationships)
        sparkdf = spark.createDataFrame(relationships)
        sparkdf = sparkdf.withColumn('WSID', lit(WSID))
        sparkdf = sparkdf.withColumn('SMID', lit(dataset_ID))
        sparkdf.write.format("delta").option("mergeSchema", "true").mode("append").save(f"{lh_abfs_path}/Tables/{Table_Name}")

StatementMeta(, ba97c8d8-4618-4a90-9892-4f106b56a3ce, 25, Submitted, Running, Running)

In [ ]:
#fill list of workspaces
workspaces = fabric.list_workspaces().sort_values(by='Name', ascending=True)
df = pd.DataFrame(workspaces)

# loop through the workspaces
for label, content in df.items():
    if label == 'Id': #loop through the ID's
        for id in content:
            #lijst vullen met alle items in de werkruimte
            temp_items = fabric.list_items(workspace=id)
            itemdf = pd.DataFrame(temp_items)
            #print(itemdf)
            if not itemdf.empty: # check if the list is not empty to avoid errors
            # filter out semantic models
                df_semantic_models = itemdf[(itemdf['Type'] == 'SemanticModel') & (itemdf['Display Name'] != 'Report Usage Metrics Model')]
                #print(df_semantic_models)
                for _, row in df_semantic_models.iterrows():
                    dataset_name = row['Display Name']
                    dataset_ID = row['Id']
                    #get the relationships of each model
                    try:
                        relationships = fabric.list_relationships(dataset=dataset_ID, workspace=id)
                    except Exception as e:
                        print(f"Error fetching semantic model relationships for {dataset_name}: {e}")
                        continue
                    relationships = pd.DataFrame(relationships)
                    if not relationships.empty: # check if the list is not empty to avoid errors
                        relationships = fnc_PrepareColumns(relationships)
                        sparkdf = spark.createDataFrame(relationships)
                        sparkdf = sparkdf.withColumn('WSID', lit(id))
                        sparkdf = sparkdf.withColumn('SMID', lit(dataset_ID))
                        sparkdf.write.format("delta").option("mergeSchema", "true").mode("append").save(f"{lh_abfs_path}/Tables/{Table_Name}")

                    

In [ ]:
prm_endtime = datetime.utcnow()
fnc_LognotebookRefresh(NotebookName, prm_starttime, prm_endtime)

StatementMeta(, , -1, Waiting, , Waiting)